In [5]:
from FUNCTIONS import *
%matplotlib inline

import sklearn.discriminant_analysis
random.seed(42)

In [6]:
# MAIN
data_s = np.array(pd.read_csv('data/data_s_180k').iloc[:, 1:])
data_b = np.array(pd.read_csv('data/data_b_180k').iloc[:, 1:])

# Slice & Save
events_no = int(1e5)
events_no = 2000
data_s = data_s[0:events_no*40, 0:40]
data_b = data_b[0:events_no*40, 0:40]

# # Save Datasets
# pd.DataFrame(data_s).to_csv('data_s_1000')
# pd.DataFrame(data_b).to_csv('data_b_1000')

train_examples, train_labels, val_examples, val_labels, test_examples, test_labels = preprocess_ML_sklearn(data_s, data_b)




ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\johng\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-1159c73f2f91>", line 2, in <module>
    data_s = np.array(pd.read_csv('data/data_s_180k').iloc[:, 1:])
  File "C:\Users\johng\anaconda3\lib\site-packages\pandas\io\parsers.py", line 676, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\johng\anaconda3\lib\site-packages\pandas\io\parsers.py", line 448, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "C:\Users\johng\anaconda3\lib\site-packages\pandas\io\parsers.py", line 880, in __init__
    self._make_engine(self.engine)
  File "C:\Users\johng\anaconda3\lib\site-packages\pandas\io\parsers.py", line 1114, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "C:\Users\johng\anaconda3\lib\site-packages\pandas\io\parsers.py", line 1891, in __init__
    self

FileNotFoundError: [Errno 2] File data/data_s_180k does not exist: 'data/data_s_180k'

In [ ]:
random_forest = RandomForestClassifier(random_state=42, n_jobs=-1)
svm = SVC(random_state=42)
KNN = sklearn.neighbors.KNeighborsClassifier()
naive_bayes = MultinomialNB()
BDT = sklearn.ensemble.GradientBoostingClassifier()
bagging = BaggingClassifier(random_state=42, n_jobs=-1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from time import time


# Naive Bayes
alpha = [0, 0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 5, 10, 15, 50]

# Bagging
n_estimators_bagging = [10, 20, 50, 100, 200, 300]
max_samples_bagging = [0.1, 0.2, 0.5, 1.0, 2.0, 3.0]
max_features_bagging = [0.5, 1.0, 2.0, 3.0, 50.0, 200]

# Random Forests
bootstrap = [True, False]
criterion = ['gini', 'entropy']
n_estimators_rf = [10, 50, 100, 200, 500, 1000, 2000, 5000]
max_depth_rf = [10, 20, 40, 50, 70, 100, None]
max_features_rf = ['auto', 'sqrt']
min_samples_leaf_rf = [1, 2, 4, 8, 15, 40]
min_samples_split_rf = [2, 5, 10, 30, 50]

# SVC
kernel = ['rbf']
c_parameter = [0.1, 1, 10]
gamma = [0.001, 0.01, 0.1, 1, 5]


# KNN
n_neighbors=[1, 2, 5, 10, 20, 30]
leaf_size_KNN=[1, 5, 10, 20, 30, 50]
p=[1, 2]

# GradientBoosting
n_estimators_GB = [10, 50, 100, 500, 2000]
learning_rate_GB = [0.01, 0.1, 1, 3]
min_samples_split_GB = [1, 2, 10]
min_samples_leaf_GB = [1, 5, 10]
max_depth_GB = [1, 3, 10, 50]



# Hyperparameters
naive_bayes_parameters = {'alpha': alpha}
bagging_parameters = {'n_estimators': n_estimators_bagging, 'max_features': max_features_bagging, 'max_samples': max_samples_bagging}
random_forest_parameters = {'bootstrap': bootstrap, 'criterion': criterion, 'max_features':max_features_rf, 'n_estimators': n_estimators_rf, 'max_depth': max_depth_rf, 'min_samples_leaf': min_samples_leaf_rf, 'min_samples_split': min_samples_split_rf}
svm_parameters = {'kernel': kernel, 'C': c_parameter, 'gamma': gamma}
GB_parameters = {'n_estimators_': n_estimators_GB, 'learning_rate': learning_rate_GB, 'min_samples_split': min_samples_split_GB, 'min_samples_split': min_samples_split_GB, 'min_samples_leaf': min_samples_leaf_GB, 'max_depth': max_depth_GB}
KNN_parameters = {'n_neighbors': n_neighbors, 'leaf_size': leaf_size_KNN, 'p': p}

# Scoring object using accuracy
scorer = make_scorer(accuracy)


clfs_param =[(naive_bayes, naive_bayes_parameters), 
             (random_forest, random_forest_parameters),
             (KNN, KNN_parameters),
             (BDT, GB_parameters),
             (bagging, bagging_parameters),
             (svm, svm_parameters)]

In [ ]:
# Create list to store models
models = []
unopt_accuracy = []
accuracies = []

# Perform grid search
for clf, parameter in clfs_param:
    print('\n{}\n'.format(clf.__class__.__name__))
    
    grid_obj = GridSearchCV(clf, parameter, scoring=scorer, n_jobs = 6)
    
    # Perform grid search
    start = time.time()
    grid_fit = grid_obj.fit(train_examples, train_labels)
    end = time.time()
    print('Time to tune: {}s'.format(round(end - start), 2))
    
    # Get best estimator
    best_clf = grid_fit.best_estimator_
    models.append(best_clf)
    
    # Make predictions using the unoptimized and model
    start = time.time()
    predictions = (clf.fit(train_examples, train_labels)).predict(val_examples)
    best_predictions = best_clf.predict(val_examples)
    
    predictions_train = (clf.fit(train_examples, train_labels)).predict(train_examples)
    best_predictions_train = best_clf.predict(train_examples)
    end = time.time()
    print('Time to fit-predict: {}s\n'.format(round(end - start), 2))
    
    # Check hyperparameters
    print('Unoptimised: {}\n'.format(clf.get_params(deep = True)))
    print('Optimised: {}\n'.format(best_clf.get_params(deep = True)))
    
    # Print Results
    print("\nUnoptimised-accuracy-training: {:.4f}".format(accuracy_score(train_labels, predictions_train)))
    print("Optimised-accuracy-training: {:.4f}".format(accuracy_score(train_labels, best_predictions_train)))
    
    print("\nUnoptimised-accuracy-validation: {:.4f}".format(accuracy_score(val_labels, predictions)))
    print("Optimised-accuracy-validation: {:.4f}".format(accuracy_score(val_labels, best_predictions)))
    
    print('\n\n=============================================================================================')
    
    unopt_accuracy.append(accuracy_score(val_labels, predictions))
    accuracies.append(accuracy_score(val_labels, best_predictions))
    
print('All unoptimised f1 (validation): {}'.format(unopt_f1))
print('Best unoptimised f1 (validation): {}\n'.format(max(unopt_f1)))
print('All optimised f1 (validation): {}'.format(f1))
print('Best optimised f1 (validation): {}'.format(max(f1)))